# Test 010424


In [ ]:
import pandas as pd

In [ ]:
def read_data(file_path):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            # 过长的评论文本会超出io限制报错。 暂时忽略
            try:
                # 字符串预处理
                # 布尔替换成python的习惯
                str_text = line.replace("true", "True")
                str_text = str_text.replace("false", "False")
                # 转成字典形式
                raw_sample = eval(str_text)
                data.append([raw_sample['reviewerID'],
                             raw_sample['asin'],
                             raw_sample['overall'],
                             raw_sample['reviewText']])
            except:
                pass
    return data


In [ ]:
data = read_data("./data/raw/Beauty_5.json")
data_df = pd.DataFrame(data)
data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']
data = data_df["reviewText"].tolist()

In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

model_name='bert-base-uncased'
# Khởi tạo tokenizer và model BERT
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
def extract_bert_features(text, model, tokenizer, output_dim=10):


    # Tokenize văn bản và thêm token '[CLS]' và '[SEP]'
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    # Feed-forward văn bản qua model BERT để trích xuất đặc trưng
    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy đặc trưng từ lớp embedding (lớp cuối cùng trước lớp softmax)
    features = outputs.last_hidden_state

    # Pooling các đặc trưng từ các token thành một vector đặc trưng (ở đây ta sử dụng mean pooling)
    pooled_features = torch.mean(features, dim=1)  # Kích thước: (1, hidden_size)

    # Giảm chiều feature đầu ra bằng cách sử dụng một lớp Linear
    linear_layer = torch.nn.Linear(pooled_features.shape[1], output_dim)
    reduced_features = linear_layer(pooled_features)

    return reduced_features

# Tính cosine similarity
def cosine_similarity(vec1, vec2):
    return F.cosine_similarity(vec1, vec2).item()

# Tính Euclidean distance
def euclidean_distance(vec1, vec2):
    return torch.dist(vec1, vec2).item()



In [ ]:
# # Sử dụng hàm để trích xuất đặc trưng từ văn bản
# text = "Hello"
# features = extract_bert_features(text, model, tokenizer)
# print(features)

# # Sử dụng hàm để trích xuất đặc trưng từ văn bản
# text = "Hello"
# features = extract_bert_features(text, model, tokenizer)
# print(features)

# Ví dụ về sử dụng
text1 = "Hello, how are you?"
text2 = "Hi, how are you doing?"
features1 = extract_bert_features(text1, model, tokenizer)
features2 = extract_bert_features(text2, model, tokenizer)


print("Cosine similarity:", cosine_similarity(features1, features2))
print("Euclidean distance:", euclidean_distance(features1, features2))

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, Dataset

# Tạo dataset đơn giản
train_data = [
    {'text': 'This is a positive sentence.', 'label': 1},
    {'text': 'I love using BERT for natural language processing tasks.', 'label': 1},
    {'text': 'Negative sentiment is not good for the mood.', 'label': 0},
    {'text': 'The quick brown fox jumps over the lazy dog.', 'label': 1},
    {'text': 'I am not feeling well today.', 'label': 0}
]

# Chuẩn bị dữ liệu huấn luyện
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = self.data[idx]['label']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label)
        }

# Chuẩn bị mô hình BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Fine-tuning mô hình
train_dataset = CustomDataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 3
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

# Lưu mô hình sau khi fine-tune
model.save_pretrained("fine_tuned_bert_model")


In [ ]:
# Load mô hình và tokenizer đã fine-tune
model = BertForSequenceClassification.from_pretrained("fine_tuned_bert_model")
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Chuẩn bị câu văn bản để test
text = "This is a positive sentence."

# Tokenize câu văn bản
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Phân loại hoặc trích xuất đặc trưng từ câu văn bản
with torch.no_grad():
    outputs = model(**inputs)

# Nếu muốn phân loại, sử dụng logits
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print("Predicted class:", predicted_class)

# Nếu muốn trích xuất đặc trưng, sử dụng đặc trưng từ lớp embedding (lớp cuối cùng trước lớp softmax)
features = outputs.last_hidden_state.mean(dim=1).squeeze(0)  # Sửa đoạn này
print("Extracted features:", features)

In [ ]:
import pandas as pd

data = read_data("/content/drive/MyDrive/Recommandation System/AMAZON_FASHION_5.json")
data_df = pd.DataFrame(data)
data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']
data = data_df["reviewText"].tolist()
# In kết quả xuống hàng mỗi record
for record in data[:20]:  # Chỉ in 20 record đầu tiên
    print(record)

In [ ]:
from nltk.corpus import wordnet, stopwords
from collections import Counter
import re
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

def preprocess_text(text):
    # Loại bỏ ký tự không cần thiết và chuyển đổi thành chữ thường
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def get_common_words(texts, n):
    # Tiền xử lý văn bản và loại bỏ từ dừng
    preprocessed_texts = [remove_stopwords(preprocess_text(text)) for text in texts]

    # Tách thành các từ riêng lẻ
    words = ' '.join(preprocessed_texts).split()

    # Đếm tần suất của các từ
    word_counts = Counter(words)

    # Chọn các từ có tần suất xuất hiện cao nhất
    common_words = [word for word, count in word_counts.most_common(n)]

    return common_words

def convert_records_to_dataset(records):
    dataset_words = []
    for record in records:
        # Tiền xử lý văn bản
        preprocessed_text = preprocess_text(record)

        # Loại bỏ các từ dừng
        filtered_words = remove_stopwords(preprocessed_text)

        # Thêm các từ vào dataset_words
        words = filtered_words.split()
        for word in words:
          dataset_words.append(word)

    return dataset_words

def semantic_similarity(word1, word2):
    # Tìm các synset (nhóm đồng nghĩa) của từ 1 và từ 2
    synsets1 = wordnet.synsets(word1)
    synsets2 = wordnet.synsets(word2)
    print(synsets1)
    print(synsets2)
    max_similarity = 0

    # Lặp qua các synset của từ 1 và từ 2 để tìm độ tương đồng cao nhất
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.wup_similarity(synset2)  # Sử dụng phương pháp Wu-Palmer để tính độ tương đồng
            if similarity is not None and similarity > max_similarity:
                max_similarity = similarity

    return max_similarity
print(semantic_similarity("great", "best"))
# Ví dụ dữ liệu văn bản
dataset_words = convert_records_to_dataset(data[:20])
print(dataset_words)

# Lấy 10 từ xuất hiện nhiều nhất trong tập dữ liệu
common_words = get_common_words(data[:20], 10)

# Tìm các từ đồng nghĩa với từng từ phổ biến
synonyms_dict = {}
for word in common_words:
    synonyms = find_synonyms_in_dataset(word, dataset_words)
    synonyms_dict[word] = synonyms

# In kết quả
for word, synonyms in synonyms_dict.items():
    print(f"Synonyms of '{word}': {', '.join(synonyms)}")


In [ ]:
!pip install pytesseract
!pip install pillow==10.0.0
!pip install pillow-heif


In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

# Tải xuống tập tin dữ liệu ngôn ngữ cho tiếng Việt
!wget https://github.com/tesseract-ocr/tessdata/raw/main/vie.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/



In [ ]:
import os

# Thiết lập biến môi trường TESSDATA_PREFIX
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata/'

In [ ]:
!pip install easyocr

In [ ]:
import pytesseract
from PIL import Image
import matplotlib.pyplot as plt
from pillow_heif import register_heif_opener
import easyocr


register_heif_opener()
def ocr_vietnamese_text(image_path):
    # Mở ảnh sử dụng Pillow
    img = Image.open(image_path)

    # Chuyển đổi ảnh sang định dạng PNG (nếu không phải là PNG)
    if img.format != "PNG":
        img = img.convert("RGB")  # Chuyển đổi sang ảnh RGB (định dạng phổ biến nhất)
        png_image_path = "converted_image.png"
        img.save(png_image_path, format="PNG")  # Lưu ảnh dưới dạng PNG

    else:
        png_image_path = image_path  # Sử dụng ảnh gốc nếu đã là định dạng PNG
    plt.imshow(img)
    plt.show()
    # Sử dụng Tesseract OCR để chuyển đổi ảnh thành văn bản
    text = pytesseract.image_to_string(png_image_path, lang='vie')

    return text

def ocr_vietnamese_and_english(image_path):
    # Đường dẫn đến ảnh
    reader = easyocr.Reader(['en', 'vi'])  # Chỉ định ngôn ngữ cần nhận dạng
    img = Image.open(image_path)
    # Chuyển đổi ảnh sang định dạng PNG (nếu không phải là PNG)
    if img.format != "PNG":
        img = img.convert("RGB")  # Chuyển đổi sang ảnh RGB (định dạng phổ biến nhất)
        png_image_path = "converted_image.png"
        img.save(png_image_path, format="PNG")  # Lưu ảnh dưới dạng PNG

    else:
        png_image_path = image_path  # Sử dụng ảnh gốc nếu đã là định dạng PNG
    plt.imshow(img)
    plt.show()

    # Nhận dạng văn bản từ ảnh
    result = reader.readtext(png_image_path)

    # Tạo một đoạn văn bản từ kết quả nhận dạng
    text = '\n'.join([entry[1] for entry in result])

    return text

# Đường dẫn đến ảnh cần chuyển đổi
image_path = "/content/drive/MyDrive/Recommandation System/IMG_9218.heic"

# Sử dụng hàm ocr_vietnamese_text để chuyển đổi ảnh sang văn bản
# text = ocr_vietnamese_text(image_path)
text = ocr_vietnamese_and_english(image_path)

# In văn bản được chuyển đổi
print(text)

# **Test 080424**

In [213]:
import pandas as pd
import json

In [214]:
def read_data(file_path):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            # 过长的评论文本会超出io限制报错。 暂时忽略
            try:
                # 字符串预处理
                # 布尔替换成python的习惯
                str_text = line.replace("true", "True")
                str_text = str_text.replace("false", "False")
                # 转成字典形式
                raw_sample = eval(str_text)
                data.append(raw_sample)
            except:
                pass
    return data

def remove_duplicate_rows(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'asin', 'reviewerID'])
    return df

def remove_duplicate_rows_ver2(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'itemID', 'reviewerID'])
    return df

In [215]:
data = read_data("/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/All_Beauty_5.json")
data_df = pd.DataFrame(data)
print(data_df.head())
# data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']

data_rating = data_df["overall"].tolist()
data_review = data_df["reviewText"].tolist()
data_reviewerID = data_df["reviewerID"].tolist()
data_itemID = data_df["asin"].tolist()

   overall  verified   reviewTime      reviewerID        asin  \
0      5.0      True   09 1, 2016  A3CIUOJXQ5VDQ2  B0000530HU   
1      5.0      True  11 14, 2013  A3H7T87S984REU  B0000530HU   
2      1.0      True  08 18, 2013  A3J034YH7UG4KT  B0000530HU   
3      5.0     False   05 3, 2011  A2UEO5XR3598GI  B0000530HU   
4      5.0      True   05 6, 2011  A3SFRT223XXWF7  B00006L9LC   

                                               style     reviewerName  \
0  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...         Shelly F   
1  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...     houserules18   
2  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...             Adam   
3  {'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...           Rich K   
4                          {'Size:': ' 200ml/6.7oz'}  C. C. Christian   

                                          reviewText  \
0                   As advertised. Reasonably priced   
1  Like the oder and the feel when I put it on my...   
2 

In [216]:
print(len(data_rating))
print(len(data_reviewerID))

5264
5264


In [217]:
import pandas as pd
# step01:
def defineRating(data_rating):
  T_v = []
  for rating in data_rating:
    if rating >= 4:
      T_v.append(1)
    else:
      T_v.append(-1)
  return T_v

T_v = defineRating(data_rating)

# Define the data for the dataframe
data = {'id': range(len(T_v)), 'T_v': T_v}

# Create the dataframe
df_rating = pd.DataFrame(data)

# Print the dataframe
print(df_rating)


        id  T_v
0        0    1
1        1    1
2        2   -1
3        3    1
4        4    1
...    ...  ...
5259  5259    1
5260  5260    1
5261  5261    1
5262  5262    1
5263  5263    1

[5264 rows x 2 columns]


In [218]:
# Thêm vào dataframe df_rating cột C_v
df_rating['reviewerID'] = data_reviewerID
df_rating['asin'] = data_itemID
df_rating['overall'] = data_rating
df_rating['reviewText'] = data_review
# Loại bỏ các dòng có reviewText trùng lặp
# df_rating = remove_duplicate_rows(df_rating)
print(df_rating)

        id  T_v      reviewerID        asin  overall  \
0        0    1  A3CIUOJXQ5VDQ2  B0000530HU      5.0   
1        1    1  A3H7T87S984REU  B0000530HU      5.0   
2        2   -1  A3J034YH7UG4KT  B0000530HU      1.0   
3        3    1  A2UEO5XR3598GI  B0000530HU      5.0   
4        4    1  A3SFRT223XXWF7  B00006L9LC      5.0   
...    ...  ...             ...         ...      ...   
5259  5259    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5260  5260    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5261  5261    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5262  5262    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5263  5263    1   AUX122XW8ONG6  B01DLR9IDI      5.0   

                                             reviewText  
0                      As advertised. Reasonably priced  
1     Like the oder and the feel when I put it on my...  
2     I bought this to smell nice after I shave.  Wh...  
3     HEY!! I am an Aqua Velva Man and absolutely lo...  
4     If you ever want to feel pamper

In [7]:

# ratings_grouped = df_rating.groupby('reviewerID')['overall'].agg(['min', 'max', list])
# print(ratings_grouped)
# filtered_users_max = ratings_grouped[ratings_grouped['max'] != 5.0]
# print(filtered_users_max)

# user_ratings = df_rating[df_rating['reviewerID'] == 'A2I9O5E0Q731GN']['overall'].tolist()
# print(user_ratings)




                min  max             list
reviewerID                               
A11QGZ39A7ZF0X  5.0  5.0  [5.0, 5.0, 5.0]
A24HQ2N7332W7W  5.0  5.0            [5.0]
A24W4W9E62FZP2  2.0  2.0            [2.0]
A2G90R2ZU6KU5D  5.0  5.0            [5.0]
A2UEO5XR3598GI  5.0  5.0            [5.0]
A3CIUOJXQ5VDQ2  5.0  5.0            [5.0]
A3H7T87S984REU  5.0  5.0            [5.0]
A3J034YH7UG4KT  1.0  1.0            [1.0]
A3SFRT223XXWF7  5.0  5.0            [5.0]
A7ID5H7FWLJHC   1.0  1.0            [1.0]
AYKOSAJTP5AVS   1.0  1.0            [1.0]
                min  max   list
reviewerID                     
A24W4W9E62FZP2  2.0  2.0  [2.0]
A3J034YH7UG4KT  1.0  1.0  [1.0]
A7ID5H7FWLJHC   1.0  1.0  [1.0]
AYKOSAJTP5AVS   1.0  1.0  [1.0]
[]


# SODCM

In [ ]:
from textblob import TextBlob


# Define function to extract a string's sentiment
find_sentiment = lambda text: TextBlob(text).sentiment[0]
# Create new column in dataframe
C_v = data_df["reviewText"].apply(find_sentiment)



In [ ]:
# Thêm vào dataframe df_rating cột C_v
df_rating['C_v'] = C_v
print(df_rating[df_rating['id'] == 98])

In [ ]:
# step02:
import math


def euclidean_distance(p1, p2):
  distance = math.sqrt((p2 - p1)**2)
  return distance
V_d = []
for ti, ci in zip(T_v, C_v):
  V_d.append(euclidean_distance(ti, ci))
  # V_d.append((ti + ci)/2)

In [ ]:
# Thêm vào dataframe df_rating cột V_d
df_rating['V_d'] = V_d
print(df_rating)

In [ ]:
# step03:
S_pos = []
S_neg = []

i = 0
for r in data_rating:
  if r >= 4:
    S_pos.append([i, r, V_d[i]])
  else:
    S_neg.append([i, r, V_d[i]])
  i = i + 1

print(len(S_pos))
print(len(S_neg))
element = None
i = 0
for item in S_pos:
    if item[0] == 25:
        element = item
        break
    i = i + 1
# for item in S_neg:
#     if item[0] == 25:
#         element = item
#         break
print(S_pos[17])
print(element)



In [ ]:
#step04:

def calculation_IQR(S, V_d, flag='pos'):
  # Sắp xếp dữ liệu
  sorted_data = sorted(V_d)

  # Tính Q1 và Q3
  q1_index = int(len(sorted_data) * 0.25)
  q3_index = int(len(sorted_data) * 0.75)
  Q1 = sorted_data[q1_index]
  Q3 = sorted_data[q3_index]

  # Tính IQR
  IQR = Q3 - Q1

  # Tính F_L và F_U
  F_L = Q1 - 1.5 * IQR
  F_U = Q3 + 1.5 * IQR

  if flag == 'pos':
    if F_L < 0:
      O_s =  Q3 + IQR
    else:
      O_s = F_U - IQR*F_L
  else:
    if F_U > 2:
      O_s = Q1 - IQR
    else:
      O_s = Q3 - IQR*F_U

  return O_s

O_Spos = calculation_IQR(S_pos, V_d, flag = 'pos')
O_Sneg = calculation_IQR(S_neg, V_d, flag = 'neg')

print(O_Spos)
print(O_Sneg)

In [ ]:
#step05:


# S_pos: danh sách các chỉ số của các phần tử thuộc S+
# S_neg: danh sách các chỉ số của các phần tử thuộc S-
# VD: danh sách các giá trị của biến VD
# OS_pos: giá trị OS+ như trong mã script
# OS_neg: giá trị OS- như trong mã script

# Khởi tạo danh sách O+ và O-
O_pos = [0] * len(S_pos)
O_neg = [0] * len(S_neg)

# print(S_pos)
# print(S_neg)
count = 0
# Điều chỉnh danh sách O+ và O- theo điều kiện trong mã script
for i, ri in enumerate(S_pos):
  print(i, ri)
  if ri[2] >= O_Spos:
    O_pos[i] = 'yes'
    count += 1
  else:
    O_pos[i] = 'no'

for i, ri in enumerate(S_neg):
    if ri[2] <= O_Sneg:
      O_neg[i] = 'yes'
      count += 1
    else:
      O_neg[i] = 'no'

print(count)
# In ra kết quả
# print("O+:", O_pos)
# print("O-:", O_neg)


In [ ]:
df = pd.DataFrame(S_pos, columns=['id', 'rating', 'V_d'])
df['O_pos'] = O_pos
#In ra các phần tử có giá trị 'yes'
print(df[df['O_pos'] == 'yes'])


In [ ]:
#step06:
# Hàm toggle để đảo ngược giá trị của một biến
def toggle(value):
  if value == -1:
    return 1
  else:
    return -1

# Khởi tạo danh sách TV với giá trị False cho mỗi phần tử
TV_new = T_v.copy()

count_1 = 0
for i, ri in enumerate(S_pos):
    if O_pos[i] == 'yes':
      count_1 += 1
      TV_new[ri[0]] = toggle(T_v[ri[0]])

for i, ri in enumerate(S_neg):
    if O_neg[i] == 'no':
      count_1 += 1
      TV_new[ri[0]] = toggle(T_v[ri[0]])

# print(TV_new)
# Tạo danh sách D* bằng cách nối S+ và S-
print(count_1)
# D_star = S_pos + S_neg

In [ ]:
# Thêm vào dataframe df_rating cột TV_new
df_rating['TV_new'] = TV_new
print(df_rating)

In [ ]:
# Tạo DataFrame mới để lưu các hàng thỏa mãn điều kiện
outliner = pd.DataFrame(columns=df_rating.columns)

# Lặp qua các hàng của DataFrame df_rating
for index, row in df_rating.iterrows():
    if row['TV_new'] != row['T_v']:
        # Thêm hàng vào DataFrame outliner
        outliner.loc[len(outliner)] = row

# In ra DataFrame outliner
print(outliner)

# Lưu DataFrame outliner vào file CSV
outliner.to_csv('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/outliner.csv', index=False)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Load pre-trained model and tokenizer for Amazon reviews
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'  # Example model, you can choose other pre-trained models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)




In [ ]:
#Define the text to analyze
text = "I love the product"
def sentiment_analysis(text, model, tokenizer):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt')

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted probabilities for each class
    probs = softmax(outputs.logits, dim=1).squeeze()

    # Convert the probabilities to human-readable labels
    labels = [1, 2, 3, 4, 5]  # For this model, it predicts star ratings
    predicted_label = labels[torch.argmax(probs).item()]
    # print(predicted_label)
    return predicted_label

# Print the result
print("Predicted sentiment:", sentiment_analysis(text, model, tokenizer))
print("Predicted sentiment:", torch.sigmoid(outputs.logits).squeeze()[1].item())
# print("Confidence:", probs[torch.argmax(probs)].item())

In [ ]:

predict_rating = df_rating['reviewText'].apply(lambda x: sentiment_analysis(x[:512], model=model, tokenizer=tokenizer))

predict_rating_list = predict_rating.tolist()
predict_rating_list =  defineRating(predict_rating_list)
print(predict_rating_list)

df_rating['predict_rating'] = predict_rating_list
print(df_rating)


In [ ]:
print(sentiment_analysis("The color pattern and fit is what I liked the most what I liked the least is that they are not easy to clean and stains do not come out very easy or at all", model, tokenizer))

In [ ]:
i = 0

for row in df_rating.itertuples():
    if row.predict_rating != row.T_v:
        i += 1
        print(row.predict_rating)
        print(row.T_v)
        print(row.overall)
        print(row.reviewText)
        print('------------------------')
print('count: ', i)

# Tạo DataFrame mới để lưu các hàng thỏa mãn điều kiện
outliner = pd.DataFrame(columns=['predict_rating', 'T_v', 'overall', 'reviewText'])
# Lặp qua các hàng của DataFrame df_rating
# Lặp qua các hàng của DataFrame df_rating
for index, row in df_rating.iterrows():
    if row['predict_rating'] != row['T_v']:
        # Thêm hàng vào DataFrame outliner
        outliner.loc[len(outliner)] = row

# In ra DataFrame outliner
print(outliner)

# Lưu DataFrame outliner vào file CSV
outliner.to_csv('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/outliner_bert.csv', index=False)

# BERT

In [219]:
import nltk
import math
import spacy
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import TextClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')


a = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']
brands = ['dove', 'aveda', 'silver']
keywords = ['overall', 'summary']
tagger = Classifier.load('pos')
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")
def remove_stopwords(sentence):
    """
    Loại bỏ các từ dừng từ câu sử dụng NLTK.

    Parameters:
    - sentence (str): Câu cần loại bỏ từ dừng.

    Returns:
    - str: Câu sau khi loại bỏ các từ dừng.
    """
    # Tách câu thành các từ
    words = word_tokenize(sentence)

    # Tải danh sách từ dừng từ NLTK
    stop_words = set(stopwords.words('english'))

    # Loại bỏ các từ dừng
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Ghép lại các từ thành câu mới
    filtered_sentence = ' '.join(filtered_words)

    return filtered_sentence

# df_rating['reviewText'] = df_rating['reviewText'].apply(remove_stopwords)

def get_sentiment_words(text):
    # Khởi tạo SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách câu thành các từ
    words = nltk.word_tokenize(text)

    # Lấy danh sách các từ ngừng (stop words)
    stop_words = set(stopwords.words('english'))

    # Danh sách các từ có cảm xúc
    sentiment_words = []

    # Kiểm tra từng từ trong câu
    for word in words:
        print(word, " - ", sia.polarity_scores(word)['compound'])
        # Nếu từ không phải là từ ngừng và có điểm tính cảm từ VADER lớn hơn 0
        # if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] > 0:
        if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] != 0:
            sentiment_words.append(word)

    return sentiment_words

def analysis_meaning_flair(sentence):

    # load model
    tagger = Classifier.load('frame')

    # make English sentence
    sentence = Sentence(sentence)

    # predict NER tags
    tagger.predict(sentence)

    # go through tokens and print predicted frame (if one is predicted)
    for token in sentence:
        print(token)

def sentiment_analysis_flair(sentence):
    # Kiểm tra xem câu có rỗng không
    if not sentence.strip():
        return 0.0
    # Load pretrained sentiment model
    classifier = TextClassifier.load('en-sentiment')

    # Create a Flair Sentence
    flair_sentence = Sentence(sentence)

    # Predict sentiment
    classifier.predict(flair_sentence)
    # print(flair_sentence)
    # Get sentiment value (POSITIVE/NEGATIVE) and score
    sentiment_value = flair_sentence.labels[0].value
    sentiment_score = flair_sentence.labels[0].score
    score = 0.0
    bias = 1.0
    for keyword in keywords:
        if keyword.lower() in sentence.lower():
            bias = 1.75
    
    if(sentiment_value == 'POSITIVE'):
        score = sentiment_score * bias
    else:
        score = -sentiment_score * bias

    return score

#Define the text to analyze

def sentiment_analysis(text, model, tokenizer):
    # Tokenize the text
    # print(text)
    if not text:
        return -1
    text = text[:512]
    inputs = tokenizer(text, return_tensors='pt')

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted probabilities for each class
    probs = softmax(outputs.logits, dim=1).squeeze()
    # print(probs)
    # Convert the probabilities to human-readable labels
    labels = [1, 2, 3, 4, 5]  # For this model, it predicts star ratings
    
    predicted_label = labels[torch.argmax(probs).item()]
    # print(predicted_label)
    return predicted_label

def check_string(input_value):
    # Kiểm tra xem giá trị có phải là chuỗi hay không
    if isinstance(input_value, str):
        # Nếu là chuỗi, trả về giá trị không thay đổi
        return input_value
    else:
        # Nếu không phải là chuỗi, trả về chuỗi rỗng
        return ''

def seperate_sentences(text):
    text = check_string(text)
    doc = nlp(text)
    # Khởi tạo một danh sách để lưu trữ các câu
    sentences = []
    current_sentence = ""
    # Duyệt qua mỗi từ trong văn bản
    for token in doc:
        # Nếu từ hiện tại không phải là dấu kết thúc câu, thêm từ vào câu hiện tại
        
        if token.is_sent_start:
            # Nếu có câu hiện tại, thêm câu vào danh sách câu
            if current_sentence:
                sentences.append(current_sentence.strip())
            # Bắt đầu một câu mới
            current_sentence = token.text
        elif token.text in ["but", "and"]:
            # print("that but:", current_sentence)
            if (brand.lower() in current_sentence.lower() for brand in brands):
                # print("need continue: ", current_sentence)
                current_sentence += " " + token.text
                continue
            elif current_sentence:
                sentences.append(current_sentence.strip())
                current_sentence = token.text
        else:
            # Thêm từ vào câu hiện tại
            current_sentence += " " + token.text
    # Thêm câu cuối cùng vào danh sách câu
    if current_sentence:
        sentences.append(current_sentence.strip())
    return sentences

def split_sentences_and_filter_sentiment(text):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách đoạn văn thành các câu nhỏ
    # sentences = nltk.sent_tokenize(text)
    sentences = seperate_sentences(text)
    if len(sentences) == 1 and not re.search(r'[.!?]', text):
        sentences = [text]
    # print(sentences)
    # Lọc các câu không chứa bất kỳ từ miêu tả cảm xúc nào
    filtered_sentences = []
    for sentence in sentences:
        # print("---- sentence ------")
        if not sentence.strip():
            continue
        sentence = sentence.strip()
        words = nltk.word_tokenize(sentence)
        
        # for word in words:
        #     if(sia.polarity_scores(word)['compound'] != 0 ):
        #         print(word, ":", sia.polarity_scores(word)['compound'])

        contains_sentiment = any(sia.polarity_scores(word)['compound'] != 0 for word in words)
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
        # Flair
        
        sentence_flair = Sentence(sentence)
        tagger.predict(sentence_flair)
        # print(sentence_flair.tokens)
        is_a = False
        for token in sentence_flair.tokens:
            for label in token.get_labels('pos'):
                if label.value in a:
                    # print(token, label.value)
                    is_a = True
                    break
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
                is_a = False
        #--------------
        if contains_sentiment:
            filtered_sentences.append(sentence)
        elif is_a:
            filtered_sentences.append(sentence)
        else:
            print("remove: ", sentence)
            # print(evaluate_sentiment_flair(sentence))
        # print("---- end sentence ------")
    # Thêm câu vào danh sách nếu không có câu nào chứa từ miêu tả cảm xúc
    if not filtered_sentences:
        filtered_sentences = [""]
    # print(filtered_sentences)
    return filtered_sentences

[nltk_data] Downloading package punkt to /home/triet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/triet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/triet/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


2024-04-23 08:16:13,286 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [220]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Load pre-trained model and tokenizer for Amazon reviews
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'  # Example model, you can choose other pre-trained models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

In [196]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

predictRating = []
text = "If they discontinued this scent I would go postal. <3"
# text = remove_stopwords(text)


sentences = split_sentences_and_filter_sentiment(text)
total_sentiment = 0
for sentence in sentences:
    # sentence = remove_stopwords(sentence)
    score = sentiment_analysis_flair(sentence)
    print(sentence, score)
    total_sentiment += score
    # print(total_sentiment)
print(total_sentiment)
if total_sentiment > 0.0:
    predictRating.append(1)
    print("Result: 1")
else:
    predictRating.append(-1)
    print("Result: -1")

remove:  <3
If they discontinued this scent , I would go postal . -0.9895524382591248
-0.9895524382591248
Result: -1


In [222]:
predictRating = []
filtered_sentences = []

for review in df_rating['reviewText']:
    sentences = split_sentences_and_filter_sentiment(review)
    filtered_sentences.append(" ".join(sentences))
    total_sentiment = 0
    for sentence in sentences:
        score = sentiment_analysis_flair(sentence)
        # print(sentence, score)
        total_sentiment += score
        # print(total_sentiment)
    # print(total_sentiment/len(sentences))
    if total_sentiment > 0.0:
        predictRating.append(1)
        print("Result: 1")
    else:
        predictRating.append(-1)
        print("Result: -1")

remove:  As advertised .
Result: 1
Result: 1
Result: -1
remove:  HEY ! !
Result: 1
Result: 1
Result: 1
remove:  It worked !
remove:  She got rid of any dandruff signs after 3 - 4 uses .
Result: 1
Result: -1
Result: -1
Result: -1
Result: 1
Result: -1
Result: 1
Result: 1
Result: -1
Result: -1
Result: -1
remove:  I tried aveda spray before but it was useless and waste of money
Result: -1
Result: 1
Result: 1
remove:  Got this product for me and   my daughter .
Result: 1
Result: 1
Result: -1
Result: 1
Result: 1
Result: 1
Result: -1
Result: 1
Result: 1
remove:  Tried a lot of options .
remove:  This is the one .
Result: 1
Result: -1
Result: -1
Result: -1
Result: -1
Result: -1
remove:  Doesnt smell
Result: -1
Result: -1
Result: -1
Result: -1
Result: -1
Result: 1
remove:  I have a shaved head and can see going through this bottle in a week or two at this pace .
remove:  BUYER BEWARE ! ! !
Result: -1
remove:  I bought two bottles and I am returning them both !
Result: -1
Result: 1
Result: 1
Res

In [223]:
print(predictRating)

df_rating['predictRating'] = predictRating
# Tạo DataFrame mới để lưu các hàng thỏa mãn điều kiện
outliner = pd.DataFrame(columns=['predictRating', 'T_v', 'overall', 'reviewText'])
# Lặp qua các hàng của DataFrame df_rating
# Lặp qua các hàng của DataFrame df_rating
for index, row in df_rating.iterrows():
    if row['predictRating'] != row['T_v']:
        # Thêm hàng vào DataFrame outliner
        outliner.loc[len(outliner)] = row

# In ra DataFrame outliner
print(outliner)

# Lưu DataFrame outliner vào file CSV
outliner.to_csv('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/outliner.csv', index=False)

[1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [224]:
df_rating['reviewText'] = filtered_sentences
print(df_rating)

        id  T_v      reviewerID        asin  overall  \
0        0    1  A3CIUOJXQ5VDQ2  B0000530HU      5.0   
1        1    1  A3H7T87S984REU  B0000530HU      5.0   
2        2   -1  A3J034YH7UG4KT  B0000530HU      1.0   
3        3    1  A2UEO5XR3598GI  B0000530HU      5.0   
4        4    1  A3SFRT223XXWF7  B00006L9LC      5.0   
...    ...  ...             ...         ...      ...   
5259  5259    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5260  5260    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5261  5261    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5262  5262    1   AUX122XW8ONG6  B01DLR9IDI      5.0   
5263  5263    1   AUX122XW8ONG6  B01DLR9IDI      5.0   

                                             reviewText  predictRating  
0                                     Reasonably priced              1  
1     Like the oder and the feel when I put it on my...              1  
2     I bought this to smell nice after I shave . Wh...             -1  
3     I am an Aqua Velva Man and ab

In [225]:
import pandas as pd
import json

# for index, row in df_rating.iterrows():
#     if row['predictRating'] != row['T_v']:
#         df_rating.drop(index, inplace=True)
# print("Dataset removed outliner: ", len(df_rating))

#Load json data to dataframe
all_beauty_data = pd.read_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/All_Beauty_5.json', lines=True)
print("Dataset: ", len(all_beauty_data))
# Loại bỏ các dòng có reviewText trùng lặp
# all_beauty_data = remove_duplicate_rows(all_beauty_data)
print("Dataset remover duplication: ", len(all_beauty_data))
all_beauty_data['reviewText'] = filtered_sentences
all_beauty_data['T_v'] = T_v
all_beauty_data['predictRating'] = predictRating
# Kiểm tra các bản ghi trong all_beauty_data mà không có trong df_rating
for index, row in all_beauty_data.iterrows():
    if row['predictRating'] != row['T_v']:
        all_beauty_data.drop(index, inplace=True)
print("Dataset removed outliner: ", len(all_beauty_data))

all_beauty_data.drop(columns=['T_v', 'predictRating'], inplace=True)

print("Filtered dataset: ", len(all_beauty_data))
all_beauty_data.to_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/All_Beauty_Filtered.json', orient='records', lines=True)

all_beauty_data_filtered = pd.read_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/All_Beauty_Filtered.json', lines=True)



Dataset:  5264
Dataset remover duplication:  5264
Dataset removed outliner:  4897
Filtered dataset:  4897


In [70]:
normal_sentence = "I received the shampoo because I was suffering from dandruff and I was like ok lets try it and I couldn't believe the result from only just one wash. it cleared out the dandruff without drying my hair out. My hair felt smooth and Fresh after just the first wash. I tried aveda spray before but it was useless and waste of money"
# filtered_sentences = remove_sentences(normal_sentence)
print("Normal sentences:", normal_sentence)
print("Filtered sentences:", filtered_sentences)
print(sentiment_analysis(normal_sentence, model, tokenizer))
# print(sentiment_analysis(filtered_sentences, model, tokenizer))
# for sentence in normal_sentence.split("."):
#     print(sentiment_analysis(sentence, model, tokenizer))
    # print(evaluate_sentiment_flair(sentence))
# print(evaluate_sentiment_flair(normal_sentence))
# print(evaluate_sentiment_flair(normal_sentence))



Normal sentences: I received the shampoo because I was suffering from dandruff and I was like ok lets try it and I couldn't believe the result from only just one wash. it cleared out the dandruff without drying my hair out. My hair felt smooth and Fresh after just the first wash. I tried aveda spray before but it was useless and waste of money
Filtered sentences: This is the first eye gel/cream that actually does what it said it was gonna do. 
I received the shampoo because I was suffering from dandruff and I was like ok lets try it and I couldn't believe the result from only just one wash. it cleared out the dandruff without drying my hair out. My hair felt smooth and Fresh after just the first wash. I tried aveda spray before but it was useless and waste of money
0
-1


[{'label': '1 star', 'score': 0.7068440914154053}]